# **Question 1**

In [75]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier

In [55]:
train_data = pd.read_csv('./titanic/train.csv')
train_data = train_data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Survived']]
sex_le = preprocessing.LabelEncoder()
embark_le = preprocessing.LabelEncoder()
sex_le.fit(['male', 'female'])
embark_le.fit(['S', 'C', 'Q'])
train_data['Age'].fillna(train_data['Age'].mean(), inplace=True)
train_data = train_data.dropna()
train_data['Sex'] = sex_le.transform(train_data['Sex'])
train_data['Embarked'] = embark_le.transform(train_data['Embarked'])
train_x = train_data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']].values
train_y = train_data[['Survived']].values.reshape((-1, ))
print('Train X shape:', train_x.shape)
print('Train Y shape:', train_y.shape)

Train X shape: (889, 7)
Train Y shape: (889,)


In [52]:
dtc = DecisionTreeClassifier()
dtc = dtc.fit(train_x, train_y)

In [71]:
test_data = pd.read_csv('./titanic/test.csv')
test_data = test_data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
test_data['Sex'] = sex_le.transform(test_data['Sex'])
test_data['Embarked'] = embark_le.transform(test_data['Embarked'])
test_data['Age'].fillna(test_data['Age'].mean(), inplace=True)
test_data['Fare'].fillna(test_data['Fare'].mean(), inplace=True)
test_x = test_data.values
test_y = pd.read_csv('./titanic/gender_submission.csv')[['Survived']].values.reshape((-1, ))
print('Test X shape:', test_x.shape)
print('Test Y shape:', test_y.shape)

Test X shape: (418, 7)
Test Y shape: (418,)


In [78]:
pred_y = dtc.predict(test_x)
accuracy = np.average(1 * (pred_y == test_y))
print('Accuracy on test set:', accuracy)

Accuracy on test set: 0.7799043062200957


# **Question 2**